In [30]:
from typing import TypedDict,Annotated,Sequence
from langchain_core.messages import BaseMessage,ToolMessage,SystemMessage,HumanMessage
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from pathlib import Path
import os

In [62]:
dotenv_path = Path('.env')
load_dotenv(dotenv_path=dotenv_path)
# api_key=os.getenv("deepseekkey")
api_key=os.getenv("gptAPIKEY")
# print(api_key)

In [39]:
class Agent(TypedDict):
    message : Annotated[Sequence[BaseMessage], add_messages]

In [64]:
filecontents=""
with open("events.txt",'a+') as file:
    file.seek(0)
    filecontents=(file.read())
filecontents

''

In [63]:
@tool
def saveevent(content:str)->str:
    """save the contents to a text file """
    with open("events.txt",'a+') as file:
        file.write(content)
    return "contents has been sucessfully been saved in events.txt"
@tool
def savecontent(content:str)->str:
    """save the contents to a text file """
    with open("contents.txt",'w+') as file:
        file.write(content)
    return "contents has been sucessfully been saved in events.txt"

@tool
def date():
    "returns today's date"
    from datetime import date
    return date.today()

tools = [saveevent,savecontent,date] 

In [ ]:
model= ChatOpenAI(model="z-ai/glm-4.5-air:free",api_key=api_key,base_url="https://openrouter.ai/api/v1").bind_tools(tools)

In [65]:
def process(state:Agent)->Agent:
    system_prompt = SystemMessage(content=f"""
    You are a scheduling assistant who process the information i give you into event : string, date-time :datetime(D/M/Y-H:M), priority : (high/medium/low) 
    - Once you process the information, you need to use the 'save' tool to save the processed information.
    - You can use the 'date' tool to get today's date
    the contents of this converastion are {filecontents}    
    """)
    userinput=HumanMessage(content=input("enter the your task"))
    finalmessage=[system_prompt]+[userinput]
    print(finalmessage)
    response=model.invoke(input=finalmessage)
    print(response.content)

In [66]:
graph=StateGraph(Agent)
graph.add_node("user",process)
graph.set_entry_point("user")
graph.add_edge("user",END)
app=graph.compile()

In [67]:
# son's birthday tomorrow  
app.invoke({'message':''})

[SystemMessage(content="\n    You are a scheduling assistant who process the information i give you into event : string, date-time :datetime(D/M/Y-H:M), priority : (high/medium/low) \n    - Once you process the information, you need to use the 'save' tool to save the processed information.\n    - You can use the 'date' tool to get today's date\n    the contents of this converastion are     \n    ", additional_kwargs={}, response_metadata={}), HumanMessage(content="son's birthday tomorrow ", additional_kwargs={}, response_metadata={})]



{'message': [HumanMessage(content='', additional_kwargs={}, response_metadata={}, id='d1a424e6-880f-4c92-8864-ced7eda6a71d')]}

In [68]:
# from IPython.display import Image,display
# display(Image(app.get_graph().draw_mermaid_png()))